In [7]:
## import the dataset
import numpy as np
import pandas as pd


In [8]:

messages = pd.read_csv('/home/venkey/Downloads/NLP-Live-main/smsspamcollection/SMSSpamCollection', sep='\t', names=['lbl', 'msg'])
messages
messages['msg'].loc[100]


"Please don't text me anymore. I have nothing else to say."

In [23]:
import nltk
import re
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/venkey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
## cleaning the text
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['msg'][i])
    review= review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = " ".join(review)
    corpus.append(review)
    

In [28]:
## creting bag of  words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
x = cv.fit_transform(corpus).toarray()
x.shape

(5572, 2500)

In [31]:
y = pd.get_dummies(messages['lbl'])
y= y.iloc[:,1].values
type(y)

numpy.ndarray

In [32]:
## train test split
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)
spam_detect_model = MultinomialNB().fit(x_train, y_train)


In [33]:
y_pred = spam_detect_model.predict(x_test)


In [35]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9856502242152466


In [36]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

       False       0.99      0.99      0.99       955
        True       0.94      0.96      0.95       160

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [43]:
## cearting tfidf model
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=2500)
x = tfidf.fit_transform(corpus).toarray()
x.shape


(5572, 2500)

In [46]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0) 


In [48]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train, y_train)

In [52]:

y_pred = model.predict(x_test) 
score = accuracy_score(y_test,y_pred)
print(score)
print(classification_report(y_test, y_pred))
            

0.979372197309417
              precision    recall  f1-score   support

       False       0.98      1.00      0.99       955
        True       1.00      0.86      0.92       160

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



## Word2vector implementation

In [2]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [5]:
vec_king = wv['king']
vec_king.shape

(300,)

In [33]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['msg'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess


In [ ]:
word = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        word.append(simple_preprocess(sent))
        

In [35]:
import gensim

In [ ]:
## lets train word2vector model
model = gensim.models.Word2Vec(word, window=5, min_count=2)
model.wv.index_to_key
model.wv.similar_by_word('kid')


In [ ]:

def avgword2vect(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

In [ ]:
from tqdm import tqdm

x_coepus = []
for word in tqdm(range(len(word))):
    x_coepus.append(avgword2vect(word[i]))

x_new = np.array(x_coepus)